## Data Generation

In [1]:
import numpy as np
from tqdm import tqdm

### Set constants for the computations

In [2]:
ar_arr = np.full((10,2), 0.0001)
area_list = np.array([0.16, 0.15, 0.14, 0.13, 0.12, 0.10, 0.08, 0.06, 0.04, 0.01]).reshape((10,1))
daisy_temp = np.full((10,2), 0.0)
ground_temp = np.full((10,1), 0.0)
S_list = np.array([417, 406, 386, 356, 318, 271, 218, 160, 113, 73]).reshape((10,1))
alb_arr = np.array([0.7, 0.2])
alb_g = 0.3
left_g = area_list - ar_arr.sum(axis = 1).reshape((10,1))
sigma = 5.67*10**(-8)
temp_arr = np.full((10,1), 0.0)
opt_temp = np.array([295, 295])
L = 0.8
R = 0.2
dr = 0.3 * area_list
G,A,B=3.8,204,2.17

### Helper functions for computations

In [3]:
def calc_albedo():
    return ((ar_arr*alb_arr).sum(axis = 1).reshape((10,1)) + (left_g*alb_g))/(area_list)

In [4]:
def update_ice():
    for i in range((alb_planet.shape[0])):
        alb_planet[i] = [0.6] if temp_arr[i] < 263.15 else alb_planet[i]

In [5]:
def fixed_pt_iteration(T, f, tol = 0.005):
    while sum(abs(T - f(T))) > tol:
        T = f(T)
    return T

def temp_fxn(T):
    return ((L*S_list*(1 - alb_planet) - A + G*sum(((T-273.15)*area_list)))/(B+G)) + 273.15

def mod_temp_fxn(T):
    return ((L*S_list*(1 - alb_planet)) + G*(sum(((T-273.15)*area_list)) - T))**(0.25)/(5.67*10**(-8))

In [6]:
update = lambda x: max([x, 0.00001])
vec_update = np.vectorize(update)
update_perc = lambda x: max([x - 0.00001, 0])
vec_update_perc = np.vectorize(update_perc)

In [7]:
def update_beta():
    beta = (1 - 0.003265*(opt_temp - daisy_temp)**2)
    mask_daisy = (daisy_temp <= 313.15)*(daisy_temp >= 278.15)
    return beta*mask_daisy

### Iterate through timesteps and save data

In [8]:
ind=0
t_vals_inc = np.zeros((1000))
ar_vals_inc = np.zeros((1000,10,2))
for ind in tqdm(range(1000)):
    L = 0.8 + ind/1000
    for i in range(1000):
        alb_planet = calc_albedo()
        temp_arr = (L*S_list*(1 - alb_planet)/sigma)**0.25
        temp_arr = fixed_pt_iteration(temp_arr, temp_fxn, 0.005)
        daisy_temp = (R*L*S_list*(alb_planet - alb_arr)/sigma + (temp_arr**4))**(0.25)
        beta = update_beta()
        del_arr = ar_arr*(beta*(left_g) -  dr)
        ar_arr += del_arr
        ar_arr = vec_update(ar_arr)
        left_g = area_list - ar_arr.sum(axis = 1).reshape((10,1))
        for i in range(10):
            left_g[i] = max(0, left_g[i])
    perc_arr = ((ar_arr)/area_list)*100
    ar_vals_inc[ind,:,:]= perc_arr
    t_vals_inc[ind] = np.sum(temp_arr*area_list)

100%|██████████| 1000/1000 [13:22<00:00,  1.25it/s]


In [9]:
from numpy import save
# define data
# save to npy file
save('ar_vals_inc.npy', ar_vals_inc)
save('temp_vals_inc.npy', t_vals_inc)

In [10]:
ind=0
t_vals_dec = np.zeros((1000))
ar_vals_dec = np.zeros((1000,10,2))
for ind in tqdm(range(1000)):
    L = 1.8 - ind/1000
    for i in range(1000):
        alb_planet = calc_albedo()
        temp_arr = (L*S_list*(1 - alb_planet)/sigma)**0.25
        temp_arr = fixed_pt_iteration(temp_arr, temp_fxn, 0.005)
        daisy_temp = (R*L*S_list*(alb_planet - alb_arr)/sigma + (temp_arr**4))**(0.25)
        beta = update_beta()
        del_arr = ar_arr*(beta*(left_g) -  dr)
        ar_arr += del_arr
        ar_arr = vec_update(ar_arr)
        left_g = area_list - ar_arr.sum(axis = 1).reshape((10,1))
        for i in range(10):
            left_g[i] = max(0, left_g[i])
    perc_arr = ((ar_arr)/area_list)*100
    ar_vals_dec[ind,:,:]= perc_arr
    t_vals_dec[ind] = np.sum(temp_arr*area_list)

100%|██████████| 1000/1000 [13:30<00:00,  1.23it/s]


In [11]:
from numpy import save
# define data
# save to npy file
save('ar_vals_dec.npy', ar_vals_dec)
save('temp_vals_dec.npy', t_vals_dec)

## Interactive widget

In [12]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
%matplotlib inline

In [13]:
ar_vals_inc = np.load('ar_vals_inc.npy')
ar_vals_dec = np.load('ar_vals_dec.npy')
inc_data = np.load("temp_vals_inc.npy")
dec_data = np.load("temp_vals_dec.npy")

In [14]:
@widgets.interact(
    Series = ["Increasing", "Decreasing"], Luminosity = (0.8, 1.8, 0.005))
def plot(Luminosity = 0.8 ,Series = "Increasing", grid=True):
    
    x = range(1, 11)
    
    if Series == "Increasing":
        ind = int((Luminosity - 0.8)*999)
    else:
        ind = 999 - int((Luminosity - 0.8)*1000)
    
    perc_inc = ar_vals_inc[ind, :, :]
    perc_dec = ar_vals_dec[ind, :, :]
    
    get_data = lambda ind: perc_inc if Series == "Increasing" else perc_dec
    curr_temp = inc_data[ind] if Series == "Increasing" else dec_data[ind]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    fig.suptitle("Temperature - Luminosity plots across Ice Albedo Model", fontsize = 'xx-large')
    
    ax1.set_ylim((0, 100))
    perc_arr = get_data(ind)
    
    ax1.bar(range(1, 11), perc_arr[:, 1], color='k', label = "Black Daisies")
    ax1.bar(range(1, 11), perc_arr[:, 0]+perc_arr[:, 1], bottom=perc_arr[:, 1], color='xkcd:sky blue', label = "White Daisies")
    ax1.legend()
    
    ax2.plot(np.arange(0.8, 1.8, 0.001), inc_data, label = "Increasing")
    ax2.plot(np.arange(1.8, 0.8, -0.001), dec_data, label = "Decreasing")
    ax2.plot(Luminosity, curr_temp, marker="o", markersize=7, label = f"L = {Luminosity}, T = {round(curr_temp, 2)}K")
    ax2.legend()
    
    ax1.set_xticks(range(1,11))
    ax1.set_yticks(range(0,101, 10))
    ax1.set_xlabel("Latitude zones", fontsize = 'large')
    ax1.set_ylabel("%age coverage", fontsize = 'large')
    ax1.set_title(f"{Series} Luminosity", fontsize = 'x-large')
    
    ax2.set_xlabel("Luminosity", fontsize = 'large')
    ax2.set_ylabel("Temperature", fontsize = 'large')
    ax2.set_yticks(range(260, 370, 10))
    ax2.set_title(f"Overall Temperature vs. Luminosity", fontsize = 'x-large')
    
    ax1.grid(grid)
    ax2.grid(grid)

interactive(children=(FloatSlider(value=0.8, description='Luminosity', max=1.8, min=0.8, step=0.005), Dropdown…